In [ ]:
!wget https://github.com/devm2024/nmt_keras/raw/master/fra.txt

In [9]:
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from string import digits
import re

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
%matplotlib inline

In [11]:
lines= pd.read_table('dataset/fra.txt', names=['eng', 'fr'])

In [12]:
print(lines[:10])
lines = lines[0:10000]

     eng            fr
0    Go.          Va !
1   Run!       Cours !
2   Run!      Courez !
3   Wow!    Ça alors !
4  Fire!      Au feu !
5  Help!    À l'aide !
6  Jump.        Saute.
7  Stop!   Ça suffit !
8  Stop!        Stop !
9  Stop!  Arrête-toi !


In [13]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fr=lines.fr.apply(lambda x: x.lower())
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.fr=lines.fr.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
remove_digits  = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fr=lines.fr.apply(lambda x: x.translate(remove_digits))

In [14]:
lines.fr = lines.fr.apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.head()

,eng,fr
0,go,START_ va _END
1,run,START_ cours _END
2,run,START_ courez _END
3,wow,START_ ça alors _END
4,fire,START_ au feu _END


In [16]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [17]:
len(all_eng_words), len(all_french_words)

(2146, 4567)

In [18]:
lenght_list=[]
for l in lines.fr:
    lenght_list.append(len(l.split(' ')))
print("Maximum length of a French sentence -", np.max(lenght_list))
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
print("Maximum length of a English sentence -", np.max(lenght_list))

Maximum length of a French sentence - 13
Maximum length of a English sentence - 5


In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
print(input_words[:50])

['COMMA', 'a', 'abandon', 'abhor', 'aboard', 'about', 'above', 'absent', 'absurd', 'accept', 'ache', 'ached', 'act', 'action', 'active', 'actor', 'acts', 'adaptable', 'addicted', 'adjust', 'admire', 'admit', 'adopted', 'adorable', 'adore', 'adores', 'adult', 'adults', 'advice', 'afford', 'afraid', 'after', 'afternoon', 'again', 'against', 'age', 'agent', 'agree', 'agreed', 'agrees', 'ahead', 'aim', 'aint', 'air', 'airs', 'alert', 'alibi', 'alive', 'all', 'allow']


In [20]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [21]:
encoder_input_data  = np.zeros(
    (len(lines.eng), 7),
    dtype='int16')
decoder_input_data = np.zeros(
    (len(lines.fr), 17),
    dtype='int16')
decoder_target_data = np.zeros(
    (len(lines.fr), 17, num_decoder_tokens),
    dtype='int16')

In [22]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.fr)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [23]:
print(encoder_input_data[9000])

[  95 2137  326    0    0    0    0]


<h1>LSTM Network</h1>

In [24]:
embedding_size = 50

In [25]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

In [26]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
encoder_states = [state_h, state_c]

In [31]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     107300      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     228350      input_4[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [ ]:
from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=20,
          validation_split=0.1, callbacks = callbacks_list)

Train on 9000 samples, validate on 1000 samples
Epoch 1/20
9000/9000 [==============================] - 93s 10ms/step - loss: 1.4050 - acc: 0.0588 - val_loss: 1.6245 - val_acc: 0.0588

Epoch 00001: val_acc improved from -inf to 0.05882, saving model to weights-improvement-01-0.06.hdf5


/usr/local/lib/python3.5/dist-packages/keras/engine/network.py:872: UserWarning: Layer lstm_3 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/20
9000/9000 [==============================] - 92s 10ms/step - loss: 1.3391 - acc: 0.0588 - val_loss: 1.6326 - val_acc: 0.0588

Epoch 00002: val_acc did not improve from 0.05882
Epoch 3/20
9000/9000 [==============================] - 94s 10ms/step - loss: 1.3359 - acc: 0.0588 - val_loss: 1.6402 - val_acc: 0.0588

Epoch 00003: val_acc did not improve from 0.05882
Epoch 4/20
4352/9000 [=============>................] - ETA: 47s - loss: 1.3281 - acc: 0.0588

In [ ]:
model.save('model.h5')
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

In [ ]:
decoder_state_input_h  = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in [4077,122,35,64, 4506]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)